In [3]:
import pickle
import hashlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#matplotlib notebook

In [5]:
feature_data = pd.read_csv("no_tiebreak_feature_data.csv")
feature_data.head()

,Unnamed: 0,match_id,server,returner,match_winner,set_winner,game_winner,pt1,pt2,pt3,pt4,pt5,pt6,deuce,ad_in,ad_out
0,0,2231275.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,2231275.0,2.0,1.0,2.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
2,2,2231275.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0
3,3,2231275.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,0.0
4,4,2231275.0,1.0,2.0,2.0,2.0,2.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,0.0,-1.0


In [6]:
matches_2011 = pd.read_csv("atp_matches_2011.csv")
matches_2012 = pd.read_csv("atp_matches_2012.csv")
matches_2013 = pd.read_csv("atp_matches_2013.csv")

In [50]:
matches_2011.rename(columns={'tourney_name':'tny_name'}, inplace=True)
matches_2012.rename(columns={'tourney_name':'tny_name'}, inplace=True)
matches_2013.rename(columns={'tourney_name':'tny_name'}, inplace=True)

matches_2011.rename(columns={'minutes':'wh_minutes'}, inplace=True)
matches_2012.rename(columns={'minutes':'wh_minutes'}, inplace=True)
matches_2013.rename(columns={'minutes':'wh_minutes'}, inplace=True)

In [51]:
matches_2011.head()

,tourney_id,tny_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2011-440,s-Hertogenbosch,Grass,32,A,20110612,1,103582,NaN,NaN,...,3.0,15.0,4.0,87.0,48.0,41.0,27.0,16.0,3.0,4.0
1,2011-440,s-Hertogenbosch,Grass,32,A,20110612,2,105053,NaN,NaN,...,2.0,5.0,1.0,70.0,45.0,31.0,11.0,10.0,2.0,4.0
2,2011-440,s-Hertogenbosch,Grass,32,A,20110612,3,104098,NaN,NaN,...,1.0,7.0,3.0,59.0,34.0,21.0,14.0,10.0,3.0,6.0
3,2011-440,s-Hertogenbosch,Grass,32,A,20110612,4,103917,NaN,NaN,...,2.0,1.0,5.0,65.0,39.0,26.0,11.0,11.0,1.0,5.0
4,2011-440,s-Hertogenbosch,Grass,32,A,20110612,5,103598,3.0,NaN,...,1.0,0.0,2.0,47.0,25.0,13.0,8.0,8.0,6.0,11.0


In [8]:
matches = pd.read_csv("tennis_archive_matches_ATP.csv")
matches.head()

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes
0,2231275,28-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Olivier Rochus,Fabio Fognini,2.0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0.0,66.0
1,2231276,28-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Robin Haase,Marin Cilic,2.0,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,0.0,141.0
2,2236280,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Marin Cilic,Andreas Seppi,1.0,SSSS;SRRRR;SSRRRSSS;RSRRSSSS;RSRSSS;SRRRR;SSRS...,6-1 6-3,0.0,71.0
3,2233792,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Alexandr Dolgopolov,Albert Ramos,1.0,SRSSRS;RRSSSRSS;RSRSSRSRSS;RRRR;SSRSS;SRSSS;SR...,6-3 7-5,0.0,83.0
4,2233791,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Carlos Berlocq,Juan Carlos Ferrero,2.0,RRRR;SRSSS;SSRRSS;RSRSSS;RRRSR;SSSRRS;RRRR.SRR...,6-1 7-6(5),0.0,150.0


In [40]:
# Create a list to store the data
winners = []
losers = []

for idx,row in matches.iterrows():
    
    if (row['winner'] == 1.0):
        winners.append(matches.at[idx,'server1'])
        losers.append(matches.at[idx,'server2'])
        
    elif (row['winner'] == 2.0):
        winners.append(matches.at[idx,'server2'])
        losers.append(matches.at[idx,'server1'])
    else:
        winners.append("")
        losers.append("")

matches['winner_name'] = winners
matches['loser_name'] = losers

        

In [62]:
matches.head(184)

,pbp_id,date,tny_name_x,tour,draw,server1,server2,winner,pbp,score_x,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2231275,28-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Olivier Rochus,Fabio Fognini,2.0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231276,28-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Robin Haase,Marin Cilic,2.0,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2236280,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Marin Cilic,Andreas Seppi,1.0,SSSS;SRRRR;SSRRRSSS;RSRRSSSS;RSRSSS;SRRRR;SSRS...,6-1 6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2236280,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Marin Cilic,Andreas Seppi,1.0,SSSS;SRRRR;SSRRRSSS;RSRRSSSS;RSRSSS;SRRRR;SSRS...,6-1 6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2236280,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Marin Cilic,Andreas Seppi,1.0,SSSS;SRRRR;SSRRRSSS;RSRRSSSS;RSRSSS;SRRRR;SSRS...,6-1 6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2233792,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Alexandr Dolgopolov,Albert Ramos,1.0,SRSSRS;RRSSSRSS;RSRSSRSRSS;RRRR;SSRSS;SRSSS;SR...,6-3 7-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2233792,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Alexandr Dolgopolov,Albert Ramos,1.0,SRSSRS;RRSSSRSS;RSRSSRSRSS;RRRR;SSRSS;SRSSS;SR...,6-3 7-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2233791,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Carlos Berlocq,Juan Carlos Ferrero,2.0,RRRR;SRSSS;SSRRSS;RSRSSS;RRRSR;SSSRRS;RRRR.SRR...,6-1 7-6(5),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2229262,28-Jul-11,CreditAgricoleSuisseOpenGstaad-ATPGstaad2011,ATP,Main,Matthias Bachinger,Julien Benneteau,2.0,SSSS;SSSS;RRRR;SSSS;SSSS;SRRRSSRSSRRSSRSS;SSSS...,6-4 6-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2228887,28-Jul-11,CreditAgricoleSuisseOpenGstaad-ATPGstaad2011,ATP,Main,Stanislas Wawrinka,Peter Luczak,1.0,SSSS;RRSSSS;SRSSRS;RRSSSS;SSSRS;SSSRRS;RSSSS;R...,6-3 7-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
matches = pd.merge(matches, matches_2011, how='left', on=['winner_name','loser_name'])
matches = pd.merge(matches, matches_2012, how='left', on=['winner_name','loser_name'])    
matches = pd.merge(matches, matches_2013, how='left', on=['winner_name','loser_name'])



In [64]:
matches.head(185)

,pbp_id,date,tny_name_x,tour,draw,server1,server2,winner,pbp,score_x,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2231275,28-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Olivier Rochus,Fabio Fognini,2.0,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231276,28-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Robin Haase,Marin Cilic,2.0,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2236280,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Marin Cilic,Andreas Seppi,1.0,SSSS;SRRRR;SSRRRSSS;RSRRSSSS;RSRSSS;SRRRR;SSRS...,6-1 6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2236280,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Marin Cilic,Andreas Seppi,1.0,SSSS;SRRRR;SSRRRSSS;RSRRSSSS;RSRSSS;SRRRR;SSRS...,6-1 6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2236280,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Marin Cilic,Andreas Seppi,1.0,SSSS;SRRRR;SSRRRSSS;RSRRSSSS;RSRSSS;SRRRR;SSRS...,6-1 6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2233792,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Alexandr Dolgopolov,Albert Ramos,1.0,SRSSRS;RRSSSRSS;RSRSSRSRSS;RRRR;SSRSS;SRSSS;SR...,6-3 7-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2233792,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Alexandr Dolgopolov,Albert Ramos,1.0,SRSSRS;RRSSSRSS;RSRSSRSRSS;RRRR;SSRSS;SRSSS;SR...,6-3 7-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2233791,29-Jul-11,ATPStudenaCroatiaOpen-ATPUmag2011,ATP,Main,Carlos Berlocq,Juan Carlos Ferrero,2.0,RRRR;SRSSS;SSRRSS;RSRSSS;RRRSR;SSSRRS;RRRR.SRR...,6-1 7-6(5),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2229262,28-Jul-11,CreditAgricoleSuisseOpenGstaad-ATPGstaad2011,ATP,Main,Matthias Bachinger,Julien Benneteau,2.0,SSSS;SSSS;RRRR;SSSS;SSSS;SRRRSSRSSRRSSRSS;SSSS...,6-4 6-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2228887,28-Jul-11,CreditAgricoleSuisseOpenGstaad-ATPGstaad2011,ATP,Main,Stanislas Wawrinka,Peter Luczak,1.0,SSSS;RRSSSS;SRSSRS;RRSSSS;SSSRS;SSSRRS;RSSSS;R...,6-3 7-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
matches.to_csv('matches_ranking_test')